# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
import findspark

In [2]:
findspark.init('/home/nowshad/spark-3.3.2-bin-hadoop3/')
from pyspark.sql import SparkSession

In [3]:
spark=SparkSession.builder.appName('LRCP').getOrCreate()

23/03/20 00:06:21 WARN Utils: Your hostname, nowshad resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/03/20 00:06:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/20 00:06:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/20 00:06:24 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/03/20 00:06:24 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [6]:
from pyspark.ml.classification import LogisticRegression 

In [8]:
data=spark.read.csv('/home/nowshad/Downloads/Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Logistic_Regression/customer_churn.csv',inferSchema=True, header=True)

In [9]:
data.show(3)

+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|           Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|   Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|     Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
only showing top 3 rows



In [10]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [11]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [47]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [17]:
from pyspark.ml.feature import VectorAssembler

In [50]:
assembler=VectorAssembler(inputCols=['Age','Total_Purchase','Account_Manager','Years','Num_Sites',],outputCol='features')

In [51]:
vec_data=assembler.transform(data)

In [52]:
vec_data.head(1)[0]

Row(Names='Cameron Williams', Age=42.0, Total_Purchase=11066.8, Account_Manager=0, Years=7.22, Num_Sites=8.0, Onboard_date=datetime.datetime(2013, 8, 30, 7, 0, 40), Location='10265 Elizabeth Mission Barkerburgh, AK 89518', Company='Harvey LLC', Churn=1, features=DenseVector([42.0, 11066.8, 0.0, 7.22, 8.0]))

In [53]:
final_data=vec_data.select('features','Churn')

In [54]:
final_data.show()

+--------------------+-----+
|            features|Churn|
+--------------------+-----+
|[42.0,11066.8,0.0...|    1|
|[41.0,11916.22,0....|    1|
|[38.0,12884.75,0....|    1|
|[42.0,8010.76,0.0...|    1|
|[37.0,9191.58,0.0...|    1|
|[48.0,10356.02,0....|    1|
|[44.0,11331.58,1....|    1|
|[32.0,9885.12,1.0...|    1|
|[43.0,14062.6,1.0...|    1|
|[40.0,8066.94,1.0...|    1|
|[30.0,11575.37,1....|    1|
|[45.0,8771.02,1.0...|    1|
|[45.0,8988.67,1.0...|    1|
|[40.0,8283.32,1.0...|    1|
|[41.0,6569.87,1.0...|    1|
|[38.0,10494.82,1....|    1|
|[45.0,8213.41,1.0...|    1|
|[43.0,11226.88,0....|    1|
|[53.0,5515.09,0.0...|    1|
|[46.0,8046.4,1.0,...|    1|
+--------------------+-----+
only showing top 20 rows



In [55]:
train_data,test_data=final_data.randomSplit([0.7,0.3])

In [56]:
log_reg=LogisticRegression(featuresCol='features',labelCol='Churn')

In [57]:
lg_model=log_reg.fit(train_data)

In [58]:
Lg_summary=lg_model.summary

In [59]:
Lg_summary.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              Churn|         prediction|
+-------+-------------------+-------------------+
|  count|                619|                619|
|   mean|0.17447495961227788|0.13893376413570274|
| stddev| 0.3798243171683688| 0.3461571193390969|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [60]:
Lg_summary.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|Churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[22.0,11254.38,1....|  0.0|[5.05588210040770...|[0.99366859830437...|       0.0|
|[25.0,9672.03,0.0...|  0.0|[5.01538605569500...|[0.99340866394025...|       0.0|
|[26.0,8787.39,1.0...|  1.0|[0.66411041025671...|[0.66018312974950...|       0.0|
|[26.0,8939.61,0.0...|  0.0|[6.86271103436953...|[0.99895501898558...|       0.0|
|[27.0,8628.8,1.0,...|  0.0|[5.88414601518898...|[0.99722450278494...|       0.0|
|[28.0,8670.98,0.0...|  0.0|[8.37141706614,-8...|[0.99976866609774...|       0.0|
|[28.0,9090.43,1.0...|  0.0|[1.61148661550472...|[0.83361768114560...|       0.0|
|[28.0,11128.95,1....|  0.0|[4.52429589264292...|[0.98927394962362...|       0.0|
|[28.0,11245.38,0....|  0.0|[3.97397458368477...|[0.98154829760841...|       0.0|
|[29.0,5900.78,1

In [65]:
pred_and_labels=lg_model.evaluate(test_data)

In [66]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|Churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[28.0,11204.23,0....|    0|[1.95755261045142...|[0.87626784458234...|       0.0|
|[29.0,13240.01,1....|    0|[7.31186562512755...|[0.99933287489962...|       0.0|
|[30.0,7960.64,1.0...|    1|[3.38164484979639...|[0.96712594069651...|       0.0|
|[30.0,11575.37,1....|    1|[4.29951233670916...|[0.98660663972917...|       0.0|
|[31.0,10058.87,1....|    0|[4.76914654090360...|[0.99158381250202...|       0.0|
|[32.0,5756.12,0.0...|    0|[4.40070531800156...|[0.98788001276046...|       0.0|
|[32.0,8575.71,0.0...|    0|[4.00686851331932...|[0.98213470577749...|       0.0|
|[32.0,9885.12,1.0...|    1|[1.86107058373523...|[0.86542168450262...|       0.0|
|[32.0,10716.75,0....|    0|[4.69489777573549...|[0.99094101362417...|       0.0|
|[32.0,12547.91,

In [43]:
results=lg_model.transform(test_data)

In [63]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [40]:
my_evel=BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Churn')

In [67]:
AUC=my_evel.evaluate(pred_and_labels.predictions)

In [68]:
AUC

0.7347081091850965

### Predict on New Data

In [69]:
final_lr_model=log_reg.fit(final_data)

In [70]:
new_customers=spark.read.csv('/home/nowshad/Downloads/Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Logistic_Regression/new_customers.csv', inferSchema=True,header=True)

In [71]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [72]:
test_new_customers=assembler.transform(new_customers)

In [73]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [74]:
final_results=final_lr_model.transform(test_new_customers)

In [76]:
final_results.select('Company', 'prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

